In [4]:
import pandas as pd
import numpy as np

data = pd.read_feather('tavbase/Sample.feather')

In [5]:
def zscore(df, grupo, var, score):
    media = df[var].mean()
    dpadr = df[var].std()
    gr = df.groupby(grupo)[var]
    dc = gr.describe()
    dc = dc.reset_index()
    dc[score] = dc['mean'].apply(lambda x: (x - media) / dpadr)
    dcz = dc[[grupo, score]]
    df = df.merge(dcz, on=grupo, how='left')
    return df

In [6]:
## Classificação zscore
dimensao = 'Customer ID'
medidas = ['Sales', 'Profit', 'Quantity']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()

for medida in medidas:
    grupo = zscore(grupo, dimensao, medida, 'z' + medida)

zmedidas = ['z'+med for med in medidas]
zgrupo = grupo[zmedidas]
grupo['score'] = zgrupo.sum(axis=1)

media_score = grupo['score'].mean()
dpadr_score = grupo['score'].std()
grupo['classe'] = grupo['score'].apply(
    lambda x: int((x - media_score) / dpadr_score) + 3
)

grupo['rank'] = grupo['score'].rank(ascending=False)
grupo['lucro'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)

grupo

,Customer ID,Sales,Profit,Quantity,zSales,zProfit,zQuantity,score,classe,rank,lucro
0,AA-10315,505.778182,-32.989318,2.727273,1.460052,-0.772648,-1.483921,-0.796517,3,500.0,0
1,AA-10375,70.426000,18.492160,2.733333,-0.827152,-0.112652,-1.475448,-2.415252,2,749.0,1
2,AA-10480,149.209333,36.318950,3.000000,-0.413249,0.115888,-1.102631,-1.399992,3,637.0,1
3,AA-10645,282.607500,47.655739,3.555556,0.287583,0.261227,-0.325931,0.222879,3,304.0,1
4,AB-10015,147.692667,21.557750,2.166667,-0.421217,-0.073351,-2.267683,-2.762251,2,767.0,1
...,...,...,...,...,...,...,...,...,...,...,...
788,XP-21865,84.809214,22.186786,3.571429,-0.751587,-0.065287,-0.303739,-1.120613,3,576.0,1
789,YC-21895,681.793750,163.203625,3.875000,2.384782,1.742557,0.120672,4.248012,5,26.0,1
790,YS-21880,560.037000,148.191025,4.833333,1.745111,1.550095,1.460481,4.755687,5,20.0,1
791,ZC-21910,258.893774,-33.295129,3.387097,0.162999,-0.776568,-0.561446,-1.175016,3,588.0,0


In [7]:
grupo.to_feather('tavbase/classificacaoz_consumidor.feather')